In [69]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import xgboost as xg
from sklearn.svm import SVR
from sklearn import preprocessing
from datetime import datetime as dt
from sklearn.model_selection import train_test_split, GroupKFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from functools import partial
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval
from hyperopt.pyll import scope as ho_scope

In [70]:
# Importar la data
data = pd.read_csv('data.csv')

In [71]:
x = data.drop('actual_productivity', axis=1)
y = data['actual_productivity']

In [72]:
scaler = preprocessing.StandardScaler().fit(x)
x_scaled = scaler.transform(x)

In [73]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=127, random_state=10)

### Modelo simple

In [74]:
# Definir el modelo SVC
model_svm = SVR()

In [75]:
# Los parámetros que el modelo utiliza son

print('Parámetros el modelo:\n')
print(model_svm.get_params())

Parámetros el modelo:

{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [76]:
start = dt.now()
# Ajustar el modelo al conjunto de datos de entrenamiento
model_svm.fit(x_train, y_train)
end = dt.now()

In [77]:
# Tiempo que se requiere para ajustar el modelo
(end - start).seconds

0

In [78]:
# Valores predichos por el modelo
y_pred_train = model_svm.predict(x_train)
y_pred_test = model_svm.predict(x_test)

# Calcular el error para el conjunto de entrenamiento y prueba
error_train = mean_absolute_error(y_train, y_pred_train)
error_test = mean_absolute_error(y_test, y_pred_test)

In [79]:
# Se observa al aplicar un modelo SVM con sus parámetros por defecto
# el MAE para el conjunto de entrenamiento es de 0.08 y en el conjunto de pruebas es de 0.0876

print('MAE del conjunto de entrenamiento: {:.7f}'.format(error_train))
print('MAE del conjunto de prueba: {:.7f}'.format(error_test))

MAE del conjunto de entrenamiento: 0.0823429
MAE del conjunto de prueba: 0.0875906


### Modelo con ajuste de hiper parámetros

#### Grid search

In [98]:
# Crear la grilla de parámetros
param_grid = {
    'C': [0.05, 10, 20, 50, 100, 200, 300]
    , 'kernel': ['rbf']
    , 'gamma': [0.05, 1, 10, 20, 50, 100]
}

In [99]:
# Definir el modelo SVC
model_svm = SVR()

In [100]:
grid_search = GridSearchCV(estimator = model_svm
             , param_grid = param_grid
             , cv = 10
             , n_jobs = -1)

In [101]:
start = dt.now()
grid_search.fit(x_train, y_train)
end = dt.now()

In [102]:
# Tiempo que se requiere para ajustar el modelo
(end - start).seconds

62

In [103]:
grid_search.best_params_

{'C': 0.05, 'gamma': 0.05, 'kernel': 'rbf'}

In [104]:
best_grid = grid_search.best_estimator_

In [105]:
joblib.dump(best_grid, 'resultados/smv_grid.pkl')

['resultados/smv.pkl']

In [106]:
# Valores predichos por el modelo
y_pred_train_grid = best_grid.predict(x_train)
y_pred_test_grid = best_grid.predict(x_test)

# Calcular el error para el conjunto de entrenamiento y prueba
error_train_grid = mean_absolute_error(y_train, y_pred_train_grid)
error_test_grid = mean_absolute_error(y_test, y_pred_test_grid)

In [107]:
# Se observa al aplicar un modelo SVM con un ajuste de hiper parámetros
# el MAE para el conjunto de entrenamiento es de 0.0421 y en el conjunto de pruebas es de 0.065

print('MAE del conjunto de entrenamiento: {:.7f}'.format(error_train_grid))
print('MAE del conjunto de prueba: {:.7f}'.format(error_test_grid))

MAE del conjunto de entrenamiento: 0.1020867
MAE del conjunto de prueba: 0.0951110


#### Optimización bayesiana

In [108]:
def optimize(search_space):
    model = SVR(**search_space)
    scores = cross_val_score(model, X=x_train, y=y_train, cv=10, scoring='neg_mean_absolute_error').mean()
    
    return {'loss':scores, 'status': STATUS_OK, 'model': model}

In [109]:
search_space = {
    'C': ho_scope.int(hp.quniform('C', 0.01, 300, 50.1))
    ,'kernel': hp.choice('kernel', ['rbf'])
    ,'gamma': ho_scope.int(hp.quniform('gamma', 0.01, 100, 0.1))
}

optimization_function = partial(
    optimize
)

In [110]:
trials = Trials()
start = dt.now()

result = fmin(
    fn = optimization_function
    , space = search_space
    , max_evals = 200
    , trials=trials
    , algo = tpe.suggest
)

end = dt.now()

 10%|████▎                                        | 19/200 [00:53<06:26,  2.14s/trial, best loss: -0.14103139880918197]

C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx",

 11%|████▉                                        | 22/200 [01:00<05:40,  1.91s/trial, best loss: -0.14103139880918197]

C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx",

 12%|█████▍                                       | 24/200 [01:00<02:57,  1.01s/trial, best loss: -0.14103139880918197]

...

 98%|███████████████████████████████████████████▎| 197/200 [02:10<00:00,  6.94trial/s, best loss: -0.14119879892975853]

C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx",

100%|███████████████████████████████████████████▊| 199/200 [02:10<00:00,  5.73trial/s, best loss: -0.14119879892975853]

C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\Erick\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx",

100%|████████████████████████████████████████████| 200/200 [02:10<00:00,  1.53trial/s, best loss: -0.14119879892975853]


In [111]:
# Tiempo que se requiere para ajustar el modelo
(end - start).seconds

130

In [112]:
print(space_eval(search_space, result))

{'C': 100, 'gamma': 99, 'kernel': 'rbf'}


In [113]:
best_model_bo = trials.best_trial['result']['model']

In [114]:
joblib.dump(best_model_bo, 'resultados/svm_bo.pkl')

['resultados/svm.pkl']

In [115]:
best_model_bo.fit(x_train, y_train)

SVR(C=100, gamma=99)

In [116]:
# Valores predichos por el modelo
y_pred_train_bo = best_model_bo.predict(x_train)
y_pred_test_bo = best_model_bo.predict(x_test)

# Calcular el error para el conjunto de entrenamiento y prueba
error_train_bo = mean_absolute_error(y_train, y_pred_train_bo)
error_test_bo = mean_absolute_error(y_test, y_pred_test_bo)

print('MAE del conjunto de entrenamiento: {:.7f}'.format(error_train_bo))
print('MAE del conjunto de prueba: {:.7f}'.format(error_test_bo))

MAE del conjunto de entrenamiento: 0.0802472
MAE del conjunto de prueba: 0.1317564


In [117]:
# trials.losses()

In [118]:
joblib.dump(trials, 'resultados/svm_bo_trials.pkl')

['resultados/svm.pkl']